In [4]:
import numpy as np
from qiskit_ibm_provider import IBMProvider
from itertools import product

In [5]:
# IBMProvider.save_account(token=<INSERT_IBM_QUANTUM_TOKEN>)
# provider = IBMProvider()
# job_id   = "cp4dpjambjng008jmwvg"
# result   = provider.retrieve_job(job_id)

In [8]:
shots = 1024
backends = ["ibm_cusco"]

for abbrev, backend in product(["BB-QCD", "TT-QCD"], backends):
    result = np.load(f"ibmq/{backend}-s{1024}-{abbrev}_r0-p_4_n25_b10-9_1_9.npy", allow_pickle=True)
    result = result.item()
    print(f"{abbrev} in backend {backend}:")
    print(f"IBMQ Frobinius norm = {np.linalg.norm(result['meas_penl']-result['meas_ibmq'])}")
    print("-"*100)

BB-QCD in backend ibm_cusco:
IBMQ Frobinius norm = 5.611314296722412
----------------------------------------------------------------------------------------------------
TT-QCD in backend ibm_cusco:
IBMQ Frobinius norm = 5.236304759979248
----------------------------------------------------------------------------------------------------
